---------------------------------------------Multiple Flight Cancellation -------------------------------------------

In [1]:
import pandas as pd

In [2]:
sch_data = 'SCH-ZZ-20231208_035117.xlsx'      #Importing schedule data
inv_data = 'INV-ZZ-20231208_041852.xlsx'      #Importing inventory data

In [3]:
sch = pd.read_excel(sch_data)
inv = pd.read_excel(inv_data)

In [4]:
df1 = pd.DataFrame(sch)         #Converting excel data into dataframe
df2 = pd.DataFrame(inv)

Selecting Flights to Cancel

In [5]:
df2['Bool'] = 1   #Creating a new column in df2 and naming it as Bool and assigning all values as 1
                  #Bool value will be 0 for the flights which are cancelled

In [10]:
df2.loc[100, ['DepartureAirport', 'ArrivalAirport']]

DepartureAirport    BLR
ArrivalAirport      HYD
Name: 100, dtype: object

In [11]:
df2.loc[200, ['DepartureAirport', 'ArrivalAirport']]

DepartureAirport    HYD
ArrivalAirport      AMD
Name: 200, dtype: object

In [12]:
import numpy as np
cancelled_flights = [100, 200]       

In [13]:
for i in cancelled_flights:
    df2.at[i, 'Bool'] = 0     #Assigning a bool value of 0 to the cancelled flights
print(cancelled_flights)

[100, 200]


In [14]:
threshold1 = pd.Timedelta(hours=6)
threshold2 = pd.Timedelta(hours=12)
threshold3 = pd.Timedelta(hours=24)
threshold4 = pd.Timedelta(hours=48)
threshold5 = pd.Timedelta(hours=72)
zero_time = pd.Timedelta(hours=0)
inf_time = pd.Timedelta(weeks=52)

Algorithm for flight scoring based on arrival, departure time, arrival, departure time etc.

In [58]:
 for j in range(len(cancelled_flights)):
    new_col = f'Flight_Score_{cancelled_flights[j]}'
    new_col2 = f'Quality Score Grade_{cancelled_flights[j]}'
    df2[new_col] = 0                 #New column in the dataframe for scoring cancelled flights
    df2[new_col2] = 'NaN'            #Each flights having a bool value of 1 will have a score 
                                     #corresponding to each cancelled flight
    
    for i in range(len(df2)):
        if df2.loc[i, 'Bool'] == 1:  #Only scoring non-cancelled flights
            if df2.loc[i, 'DepartureAirport'] == df2.loc[cancelled_flights[j], 
            'DepartureAirport'] and df2.loc[i, 'ArrivalAirport'] == df2.loc[cancelled_flights[j], 'ArrivalAirport']:
               df2.loc[i, new_col] = 40 
               diff_dep = df2.loc[i, 'DepartureDateTime'] - df2.loc[cancelled_flights[j], 'DepartureDateTime']
               diff_arr = df2.loc[i, 'ArrivalDateTime'] - df2.loc[cancelled_flights[j], 'ArrivalDateTime']
               if((diff_dep <= threshold4) and (diff_dep >= zero_time)):                            
                    df2.loc[i, new_col] = df2.loc[i, new_col] + 30                                  
                    if((diff_dep <= threshold3) and (diff_dep >= zero_time)):                       
                        df2.loc[i, new_col] = df2.loc[i, new_col] + 10   #Adding only 10 because there is a nested-if 
                        if((diff_dep <= threshold2) and (diff_dep >= zero_time)):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                            if((diff_dep <= threshold1) and (diff_dep >= zero_time)):
                                df2.loc[i, new_col] = df2.loc[i, new_col] + 20
               if((diff_arr <= threshold4) and (diff_arr >= zero_time)):                            
                    df2.loc[i, new_col] = df2.loc[i, new_col] + 30                                    
                    if((diff_arr <= threshold3) and (diff_arr >= zero_time)):                        
                        df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                        if((diff_arr <= threshold2) and (diff_arr >= zero_time)):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                            if((diff_arr <= threshold1) and (diff_arr >= zero_time)):
                                df2.loc[i, new_col] = df2.loc[i, new_col] + 20
               for k in range(len(df1)):
                    if (df2.loc[i, 'ScheduleId'] == df1.loc[k, 'ScheduleID']):
                        tail_number = df1.loc[k, 'AircraftTailNumber']
               for k in range(len(df1)):
                    if (df2.loc[cancelled_flights[j], 'ScheduleId'] == df1.loc[k, 'ScheduleID']):
                        if (df1.loc[k, 'AircraftTailNumber'] == tail_number):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 50

            #Quality grading of flights acc. to ruleset provided
            if(df2.loc[i, new_col] < 150 and df2.loc[i, new_col] > 0):
                df2.loc[i, new_col2] = 'D'
            if(df2.loc[i, new_col] >= 150):
                df2.loc[i, new_col2] = 'C'
            if(df2.loc[i, new_col] >= 180):
                df2.loc[i, new_col2] = 'B'
            if(df2.loc[i, new_col] >= 200):
                df2.loc[i, new_col2] = 'A'

df2 = pd.concat([df2], axis=1)

In [59]:
filt1 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'A')
filt2 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'B')
filt3 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'C')
filt4 = (df2['Oversold'] < 0)       #Considering only those flights which have vacant seats
Available_flights_c1 = df2.loc[(filt1 | filt2 | filt3) & filt4]

In [60]:
Available_flights_c1

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD,Bool,Flight_Score_100,Quality Score Grade_100,Flight_Score_200,Quality Score Grade_200
101,INV-ZZ-1078045,SCH-ZZ-8346723,ZZ,ZZ20240419BLRHYD3027.0,3027,BAe 146,2024-04-19,2024-04-19 18:48:00,2024-04-20 02:58:00,BLR,...,18,"{'F': 5, 'P': 4}","{'C': 7, 'J': 5, 'Z': 5}","{'Q': 8, 'R': 5, 'S': 3, 'T': 3, 'H': 5, 'M': 3}","{'Y': 3, 'A': 3, 'B': 1, 'D': 1, 'E': 1, 'G': ...",1,170,C,0,NaN


In [61]:
filt1 = (df2[f'Quality Score Grade_{cancelled_flights[1]}'] == 'A')
filt2 = (df2[f'Quality Score Grade_{cancelled_flights[1]}'] == 'B')
filt3 = (df2[f'Quality Score Grade_{cancelled_flights[1]}'] == 'C')
filt4 = (df2['Oversold'] < 0)       #Considering only those flights which have vacant seats
Available_flights_c2 = df2.loc[(filt1 | filt2 | filt3) & filt4]

In [62]:
Available_flights_c2

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD,Bool,Flight_Score_100,Quality Score Grade_100,Flight_Score_200,Quality Score Grade_200
201,INV-ZZ-7388012,SCH-ZZ-5716505,ZZ,ZZ20240518HYDAMD4335.0,4335,Airbus A321-200,2024-05-18,2024-05-18 14:19:00,2024-05-19 11:51:00,HYD,...,44,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ...",1,0,NaN,170,C


If a third flight is cancelled.....

In [325]:
F = []          # 2D Array for storing scores for each alternate flight
Av_Fl = []      # 2D Array for storing indices of each alternate flight in the inventory dataframe
V_S = []
#    F = [[f1, f2, ...], [f1', f2', ...], ...]
#Av_Fl = [[i1, i2, ...], [i1', i2', ...], ...]
#             c1              c2

#Adding scores and indices for alternate flights for each cancelled flights
for flight in range(len(cancelled_flights)):
    available_flights = globals()[f'Available_flights_c{flight+1}']  # Assuming Available_flights_c1, Available_flights_c2, etc. are defined as dataframes
    
    F_temp = []      # Temporary list for scores for the current cancelled flight
    Av_Fl_temp = []  # Temporary list for indices for the current cancelled flight
    V_S_temp = []    #Storing the available seats in each alternate flight

    for i in range(len(available_flights)):
        ix = available_flights.index[i]
        Av_Fl_temp.append(ix)
        F_temp.append(df2.loc[ix, f'Quality Score Grade_{cancelled_flights[flight]}'])
        V_S_temp.append(-df2.loc[ix, 'Oversold'])
    
    # Converting Quality Score into numerical scores
    F_temp = [4 if grade == 'A' else 3 if grade == 'B' else 2 if grade == 'C' else 1 for grade in F_temp]
    
    # Appending the temporary lists to the main lists
    F.append(F_temp)
    Av_Fl.append(Av_Fl_temp)
    V_S.append(V_S_temp)

# Now F, Av_Fl and V_S contain scores, indices and vacant seats for each cancelled flight

print(F)
print(Av_Fl)
print(V_S)

[[2], [2]]
[[101], [201]]
[[31], [76]]


Scaling down the values in V_S(Vacant Seats)

In [326]:
for j in range(len(V_S)):
    for i in range(len(V_S[j])):
        V_S[j][i] = V_S[j][i]/10
        V_S[j][i] = np.floor(V_S[j][i])   #Floor value so that no. of passengers accommodated
print(V_S)                                #in each flight is less than the flight's capacity

[[3.0], [7.0]]


In [67]:
#We have:
#F containing the scores for each alternate flight
#Av_Fl containing the index of each alternate flight in the inventory dataframe
#V_S containing the number of available seats in each alternating flight
#All the above 3 arrays have the same length. The corresponding indices represent the same flights

In [68]:
PNR = 'PNRB-ZZ-20231208_062017.xlsx'     #Importing the PNR Data
Pass = 'PNRP-ZZ-20231208_111136.xlsx'    #Importing the Passenger Data

In [69]:
PNR1 = pd.read_excel(PNR)          
Pass1 = pd.read_excel(Pass)

In [70]:
PNR_DF = pd.DataFrame(PNR1)             #Converting excel data into dataframe
Pass_DF = pd.DataFrame(Pass1)

In [71]:
Pass_DF['SSR_CODE_CD1'] = Pass_DF['SSR_CODE_CD1'].fillna(0)
Pass_DF['TierLevel'] = Pass_DF['TierLevel'].fillna(0)

Scoring Passengers

In [76]:
departure_list = PNR_DF['ORIG_CD'].to_list()
arrival_list = PNR_DF['DEST_CD'].to_list()

city_pairs = [tuple([departure_list[i], arrival_list[i]]) for i in range(len(departure_list))]

fltnum_list = PNR_DF["FLT_NUM"].to_list()

citypair_dict = {fltnum_list[i] : city_pairs[i] for i in range(len(city_pairs))}

In [77]:
paxcnt_wt = 50
connect_wt = 100
grp_threshold = 7
grp_wt = 500
class_wt = {
    'FirstClass' : 2000, 'BusinessClass' : 1750, 
    'PremiumEconomyClass' : 1500, 'EconomyClass' : 1250
}
# The airlines can vary the weights as per their rules set and have the choice to disregard certain factors as per their choice.

In [78]:
def get_recloc_score(df):
    recloc_sclist = []
    recloc_score = 0
    for i in range(len(df['RECLOC'])):
        recloc_score += df['PAX_CNT'][i]*paxcnt_wt + class_wt[df['COS_CD'][i]]
        if df['PAX_CNT'][i] >= grp_threshold:
            recloc_score += grp_wt
        recloc_sclist.append(recloc_score)
        recloc_score = 0

    df['RECLOC_SCORE'] = recloc_sclist

    return df    

#Due to multiple passengers having the same recloc, We have split calculating passenger 
#score into two parts : recloc_Score and passenger_score. The recloc score will initialise
#the pax_score variable and build up from there.

In [79]:
get_recloc_score(PNR_DF)

,RECLOC,CREATION_DTZ,DEP_KEY,ACTION_CD,COS_CD,SEG_SEQ,SEG_TOTAL,PAX_CNT,CARRIER_CD,FLT_NUM,ORIG_CD,DEST_CD,DEP_DT,DEP_DTML,ARR_DTML,DEP_DTMZ,ARR_DTMZ,RECLOC_SCORE
0,DRGS80,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,4,3,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2150
1,YEZQ47,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,2,4,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2200
2,JDDM40,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,3,2,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2100
3,AVFF16,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,4,1,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2050
4,DBLE79,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,2,2,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78179,CJEL38,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,4,7,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,2100
78180,NDXM23,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,1,9,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,2200
78181,RIUQ17,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,4,5,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,1500
78182,QLFS50,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,2,1,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,1300


In [80]:
recloc_dict = {PNR_DF['RECLOC'][i] : PNR_DF['RECLOC_SCORE'][i] for i in range(len(PNR_DF['RECLOC']))}

In [81]:
loyalty_dict = {'Platinum' : 1800, 'Gold' : 1600, 'Silver' : 1500, 0 : 0}
ssr_wt = 200
# The airlines can vary the weights as per their rules set and have the choice to disregard certain factors as per their choice.

In [82]:
def get_pnr_score(df):
    pnr_sclist = []
    pnr_score = 0
    for i in range(len(df['RECLOC'])):
        pnr_score += recloc_dict[df['RECLOC'][i]] + loyalty_dict[df['TierLevel'][i]]
        if df['SSR_CODE_CD1'][i] != 0:
            pnr_score += ssr_wt
        
        pnr_sclist.append(pnr_score)
        pnr_score = 0

    df['PNR_SCORE'] = pnr_sclist

    return df 

In [83]:
get_pnr_score(Pass_DF)

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE
0,DRGS80,2024-03-02 16:35:00,VNQ9748,Mitchell,Christina,CA,2003-06-01,1-769-506-8397,christinamitchell1@example.com,Driving License,299196000000,NaN,0,ADT,4.276771e+09,Gold,3750
1,DRGS80,2024-03-02 16:35:00,BAP1926,Parker,Rebecca,CA,2015-08-18,1-947-699-5143,rebeccaparker2@example.com,Passport,11901984430,NaN,0,CHD,8.493470e+09,Platinum,3950
2,DRGS80,2024-03-02 16:35:00,ALV9938,Swamy,Vardaniya,IN,1945-02-23,6102977873,vardaniyaswamy3@example.com,Govt-Id,4283733463,NaN,0,S65,6.731611e+09,Platinum,3950
3,YEZQ47,2024-03-02 16:35:00,CEU7075,Edwin,Stuvan,IN,2003-06-18,5791967032,stuvanedwin1@example.com,Passport,139267560,NaN,0,ADT,7.214819e+09,Platinum,4000
4,YEZQ47,2024-03-02 16:35:00,WFG9647,Hernandez,Andrea,CA,1969-07-03,+1 (378) 291-0325,andreahernandez2@example.com,Govt-Id,5459762750,NaN,0,ADT,NaN,0,2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152508,RIUQ17,2023-08-20 13:28:00,NQF8798,Shankar,Divit,IN,1981-08-22,(347) 703-8971,divitshankar3@example.com,Driving License,169860000000,NaN,0,ADT,NaN,0,1500
152509,RIUQ17,2023-08-20 13:28:00,RJI4602,Henderson,Autumn,US,2013-09-14,(371) 218-4438,autumnhenderson4@example.com,Govt-Id,7100653216,NaN,0,CHD,5.882689e+09,Gold,3100
152510,RIUQ17,2023-08-20 13:28:00,VPC0920,Ramanathan,Hridaan,IN,1944-09-06,(437) 839-4309,hridaanramanathan5@example.com,Driving License,715385534,NaN,0,S65,NaN,0,1500
152511,QLFS50,2023-08-20 13:28:00,BIB2224,Baria,Taimur,IN,2010-06-19,(771) 284-3525,taimurbaria1@example.com,Govt-Id,4431751697,NaN,0,CHD,3.701390e+09,Gold,2900


In [112]:
#Function that returns the PNRs that are affected based on flight no., and departure date of the cancelled flight
def Affected_PNR(FLT_NUM, DEP_DATE, PNR_DF):
    AffectedPNR = PNR_DF.loc[(PNR_DF['FLT_NUM'] == FLT_NUM) & (PNR_DF['DEP_DT']==DEP_DATE)]
    return AffectedPNR

In [113]:
#Dataframe consisting of affected PNRs
aff_c1 = Affected_PNR(df2.loc[cancelled_flights[0],'FlightNumber'], df2.loc[cancelled_flights[0],'DepartureDate'], PNR_DF)
aff_c2 = Affected_PNR(df2.loc[cancelled_flights[1],'FlightNumber'], df2.loc[cancelled_flights[1],'DepartureDate'], PNR_DF)
#If a 3rd flight is cancelled
#aff = Affected_PNR(df2.loc[cancelled_flights[2],'FlightNumber'], df2.loc[cancelled_flights[2],'DepartureDate'], PNR_DF)

In [114]:
#Dataframe consisting of affected passengers
aff_pax_c1 = pd.merge(Pass_DF, aff_c1[['RECLOC']], on='RECLOC', how='inner')
aff_pax_c2 = pd.merge(Pass_DF, aff_c2[['RECLOC']], on='RECLOC', how='inner')
#aff_pax_c3 = pd.merge(Pass_DF, aff_c3[['RECLOC']], on='RECLOC', how='inner')

In [115]:
print(f'{len(aff_pax_c1) + len(aff_pax_c2)} passengers are affected')

183 passengers are affected


Grouping passengers based on PNR_Score

In [116]:
def group_passengers_by_score(df):
    for i in range(len(df)):
        score = df.loc[i, 'PNR_SCORE']
        
        if score >= 3500:
            df.loc[i, 'PNR_SCORE'] = 5
        elif score >= 3000:
            df.loc[i, 'PNR_SCORE'] = 4
        elif score >= 2500:
            df.loc[i, 'PNR_SCORE'] = 3
        elif score >= 2000:
            df.loc[i, 'PNR_SCORE'] = 2
        else:
            df.loc[i, 'PNR_SCORE'] = 1
            

In [117]:
# Applying the function to each dataframe
group_passengers_by_score(aff_pax_c1)
group_passengers_by_score(aff_pax_c2)
# Add more lines if you have more dataframes

# Now aff_pax_c1 and aff_pax_c2 (and others if applicable) are modified with grouped PNR_SCORE

In [141]:
value_counts = aff_pax_c1['PNR_SCORE'].value_counts()
# Convert the Series to a DataFrame
freq_df_c1 = pd.DataFrame({'PNR_SCORE': value_counts.index, 'Frequency': value_counts.values})

value_counts = aff_pax_c2['PNR_SCORE'].value_counts()
# Convert the Series to a DataFrame
freq_df_c2 = pd.DataFrame({'PNR_SCORE': value_counts.index, 'Frequency': value_counts.values})

#value_counts = aff_pax_c3['PNR_SCORE'].value_counts()
## Convert the Series to a DataFrame
#freq_df_c3 = pd.DataFrame({'PNR_SCORE': value_counts.index, 'Frequency': value_counts.values})

In [143]:
freq_df_c2

,PNR_SCORE,Frequency
0,5,36
1,1,30
2,4,29
3,2,16
4,3,7


In [327]:
def ScoreAndFreq(df, Score, G):
    Score_temp = []
    G_temp = []
    for i in range(len(df)):
        Score_temp.append(df.loc[i, 'PNR_SCORE'])
        G_temp.append(df.loc[i, 'Frequency'])
    Score.append(Score_temp)
    G.append(G_temp)
    return Score, G

In [328]:
(Score, G) = ScoreAndFreq(freq_df_c1, [], [])
(Score, G) = ScoreAndFreq(freq_df_c2, Score, G)
#ScoreAndFreq(freq_df_c3)

In [329]:
Score 

[[4, 1, 5, 2, 3], [5, 1, 4, 2, 3]]

In [330]:
G 

[[23, 19, 17, 4, 2], [36, 30, 29, 16, 7]]

In [331]:
#Scaling down the values in G
for j in range(len(G)):
    for i in range(len(G[j])):
        G[j][i] = G[j][i]/10
        G[j][i] = np.ceil(G[j][i])   #Ceil value so that number of passengers accommodated in
                                     #a flight is always less than than the flight's capacity
print(G)

[[3.0, 2.0, 2.0, 1.0, 1.0], [4.0, 3.0, 3.0, 2.0, 1.0]]


Solving our QUBO problem using Quantum Solvers

In [160]:
#!pip install qiskit
#!pip install qiskit_algorithms
#!pip install qiskit_optimization

In [161]:
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import LinearInequalityToPenalty, QuadraticProgramToQubo

In [332]:
print(f'Av_Fl = {Av_Fl}')
print(f'V_S = {V_S}')
print(f'F = {F}')
print(f'Score = {Score}')
print(f'G = {G}')

Av_Fl = [[101], [201]]
V_S = [[3.0], [7.0]]
F = [[2], [2]]
Score = [[4, 1, 5, 2, 3], [5, 1, 4, 2, 3]]
G = [[3.0, 2.0, 2.0, 1.0, 1.0], [4.0, 3.0, 3.0, 2.0, 1.0]]


In [337]:
#Formulating the problem
qp = QuadraticProgram()

#Defining variables
for k in range(len(Score)):         #len(F) = len(Score)
    for j in range(len(F[k])):
        for i in range(len(Score[k])):
            qp.binary_var(name=f'x_{k}_{k}_{i}_{j}')

#Objective function
linear_dict = {
    f'x_{k}_{k}_{i}_{j}':F[k][j] * Score[k][i]
    for k in range(len(Score))
    for j in range(len(F[k]))
    for i in range(len(Score[k])) 
}
qp.maximize(linear=linear_dict)

#Add constraints
#Passenger Constraint (Each passenger(group here) is assigned one flight)
#for k in range(len(Score)):
#    for i in range(len(Score[k])):
#        constraint1 = {
#            f'x_{k}_{k}_{i}_{j}': 1 
#            for j in range(len(F[k]))
#        }
#        qp.linear_constraint(linear=constraint1, sense='<=', rhs=1)

#Flight Constraint (Flights shouldn't accommodate more than it can carry/fly)
for k in range(len(F)):
    for j in range(len(F[k])):
        constraint2 = {
            f'x_{k}_{k}_{i}_{j}': G[k][i] 
            for i in range(len(Score[k]))
        }
        qp.linear_constraint(linear=constraint2, sense='<=', rhs=V_S[k][j])

print(qp.prettyprint())

Problem name: 

Maximize
  8*x_0_0_0_0 + 2*x_0_0_1_0 + 10*x_0_0_2_0 + 4*x_0_0_3_0 + 6*x_0_0_4_0
  + 10*x_1_1_0_0 + 2*x_1_1_1_0 + 8*x_1_1_2_0 + 4*x_1_1_3_0 + 6*x_1_1_4_0

Subject to
  Linear constraints (2)
    3*x_0_0_0_0 + 2*x_0_0_1_0 + 2*x_0_0_2_0 + x_0_0_3_0 + x_0_0_4_0 <= 3  'c0'
    4*x_1_1_0_0 + 3*x_1_1_1_0 + 3*x_1_1_2_0 + 2*x_1_1_3_0 + x_1_1_4_0
    <= 7  'c1'

  Binary variables (10)
    x_0_0_0_0 x_0_0_1_0 x_0_0_2_0 x_0_0_3_0 x_0_0_4_0 x_1_1_0_0 x_1_1_1_0
    x_1_1_2_0 x_1_1_3_0 x_1_1_4_0



In [338]:
qaoa_mes = QAOA(sampler=Sampler(), optimizer=SPSA(), reps=1)

In [339]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

In [340]:
qaoa_result = qaoa.solve(qp)
print(qaoa_result.prettyprint())

objective function value: 36.0
variable values: x_0_0_0_0=0.0, x_0_0_1_0=0.0, x_0_0_2_0=1.0, x_0_0_3_0=0.0, x_0_0_4_0=1.0, x_1_1_0_0=1.0, x_1_1_1_0=0.0, x_1_1_2_0=0.0, x_1_1_3_0=1.0, x_1_1_4_0=1.0
status: SUCCESS


In [360]:
print("variable order:", [var.name for var in qaoa_result.variables])
X = []
for i in range(min(4, len(qaoa_result.samples))):
    sample = qaoa_result.samples[i]

    # Extract 'x' array from the sample
    x_array = sample.x
    X.append(x_array[:])   #Omitting the slack variables

variable order: ['x_0_0_0_0', 'x_0_0_1_0', 'x_0_0_2_0', 'x_0_0_3_0', 'x_0_0_4_0', 'x_1_1_0_0', 'x_1_1_1_0', 'x_1_1_2_0', 'x_1_1_3_0', 'x_1_1_4_0']


List that contains the first 4 optimal solutions


In [362]:
#Anslyzing first solution
n=1
X[n-1]

array([0., 0., 1., 0., 1., 1., 0., 0., 1., 1.])

In [363]:
aff_pax_c1['Alt_Fli'] = 'NaN'      #New column for storing the assigned alternate flight to passengers
aff_pax_c2['Alt_Fli'] = 'NaN'
#aff_pax_c3['Alt_Fli'] = 'NaN'

In [365]:
lt = len(X[n-1])
#Cancelled flight 1
lt1 = len(Score[0])*len(F[0])
for i in range(lt1):
    if X[n-1][i] == 1:
        alt_flight_number = (i // len(Score[0])) + 1
        group_index = (i % len(Score[0])) + 1
        assignment_info = f"Group {group_index} is assigned to Flight {alt_flight_number}" 
        print(assignment_info)
        condition = aff_pax_c1['PNR_SCORE'] == Score[0][group_index - 1]
        aff_pax_c1.loc[condition, 'Alt_Fli'] = Av_Fl[0][alt_flight_number - 1]  

Group 3 is assigned to Flight 1
Group 5 is assigned to Flight 1


In [370]:
#Cancelled flight 2
lt2 = len(Score[1])*len(F[1])
j = 0
for i in range(lt1, lt1 + lt2):
    if X[n-1][i] == 1:
        alt_flight_number = (j // len(Score[1])) + 1
        group_index = (j % len(Score[1])) + 1
        assignment_info = f"Group {group_index} is assigned to Flight {alt_flight_number}" 
        print(assignment_info)
        condition = aff_pax_c2['PNR_SCORE'] == Score[1][group_index - 1]
        aff_pax_c2.loc[condition, 'Alt_Fli'] = Av_Fl[1][alt_flight_number - 1] 
    j = j + 1

Group 1 is assigned to Flight 1
Group 4 is assigned to Flight 1
Group 5 is assigned to Flight 1


In [ ]:
#Cancelled flight 3
#lt3 = len(Score[2])*len(F[2])
#for i in range(lt2, lt2 + lt3):
#    j = 0
#    if X[n-1][i] == 1:
#        alt_flight_number = (j // len(Score[2])) + 1
#        group_index = (j % len(Score[2])) + 1
#        assignment_info = f"Group {group_index} is assigned to Flight {alt_flight_number}" 
#        print(assignment_info)
#        condition = aff_pax_c3['PNR_SCORE'] == Score[2][group_index - 1]
#        aff_pax_c3.loc[condition, 'Alt_Fli'] = Av_Fl[2][alt_flight_number - 1] 
#    j = j + 1

In [371]:
#Importing class from PNR Data to Passenger Data
aff_pax_c1['Class'] = 'NaN'
aff_pax_c2['Class'] = 'NaN'

for i in range(len(aff_c1)):
    recloc_aff = aff_c1.loc[aff_c1.index[i], 'RECLOC']
    class_aff = aff_c1.loc[aff_c1.index[i], 'COS_CD']

    aff_pax_c1.loc[aff_pax_c1['RECLOC'] == recloc_aff, 'Class'] = class_aff

for i in range(len(aff_c2)):
    recloc_aff = aff_c2.loc[aff_c2.index[i], 'RECLOC']
    class_aff = aff_c2.loc[aff_c2.index[i], 'COS_CD']

    aff_pax_c2.loc[aff_pax_c2['RECLOC'] == recloc_aff, 'Class'] = class_aff

#aff_pax_c3['Class'] = 'NaN'
#for i in range(len(aff_c3)):
#    recloc_aff = aff_c3.loc[aff_c3.index[i], 'RECLOC']
#    class_aff = aff_c3.loc[aff_c3.index[i], 'COS_CD']

#    aff_pax_c3.loc[aff_pax_c3['RECLOC'] == recloc_aff, 'Class'] = class_aff

In [374]:
aff_pax_c1['Alt_Fli'].value_counts()  

Alt_Fli
NaN    46
101    19
Name: count, dtype: int64

In [375]:
aff_pax_c2['Alt_Fli'].value_counts() 

Alt_Fli
NaN    59
201    59
Name: count, dtype: int64

In [376]:
aff_pax_c1['Assigned Class'] = ''     #New column for storing the newly assigned class in alternate flights
aff_pax_c2['Assigned Class'] = ''
#aff_pax_c3['Assigned Class'] = ''

In [377]:
filt = (aff_pax_c1['Alt_Fli'] != 'NaN')
Reaccommodated_List_c1 = aff_pax_c1.loc[filt] #Dataframe that contains the passengers who were rescheduled

filt = (aff_pax_c2['Alt_Fli'] != 'NaN')
Reaccommodated_List_c2 = aff_pax_c2.loc[filt] #Dataframe that contains the passengers who were rescheduled

filt = (aff_pax_c1['Alt_Fli'] == 'NaN')
Exception_List_c1 = aff_pax_c1.loc[filt] #Dataframe that contains the passengers who couldn't be rescheduled

filt = (aff_pax_c2['Alt_Fli'] == 'NaN')
Exception_List_c2 = aff_pax_c2.loc[filt] #Dataframe that contains the passengers who couldn't be rescheduled

Reaccommodated_List = pd.concat([Reaccommodated_List_c1, Reaccommodated_List_c2], ignore_index=True) 

Exception_List = pd.concat([Exception_List_c1, Exception_List_c2], ignore_index=True)

print(f'{len(Reaccommodated_List)} passengers out of {len(Reaccommodated_List) + len(Exception_List)} were reaccommodated')

78 passengers out of 183 were reaccommodated


Class Assignment

In [378]:
# Assign classes based on 'Score' and class limits
def assign_class(row):
     global class_count
     for category, limit in class_limits.items():
           if class_count[category] < limit:
               class_count[category] += 1
               return category
     return None

In [379]:
#C1

In [380]:
#Alternate flight 1
f = 1
filt = (aff_pax_c1['Alt_Fli'] == Av_Fl[0][f-1])
filtered_c1_f1 = aff_pax_c1.loc[filt]

filt = (aff_pax_c1['Alt_Fli'] == Av_Fl[0][f-1])
filtered_pass_data_c1_f1 = aff_pax_c1.loc[filt]

In [381]:
#Assigning class based on PNR scores
# Sort the DataFrame based on 'Score' in descending order
filtered_pass_data_c1_f1 = filtered_pass_data_c1_f1.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
class_limits = {
    'FirstClass': df2.loc[Av_Fl[0][f-1], 'FC_Oversold'], 
    'BusinessClass': df2.loc[Av_Fl[0][f-1], 'BC_Oversold'], 
    'PremiumEconomyClass': df2.loc[Av_Fl[0][f-1], 'PC_Oversold'], 
    'EconomyClass': df2.loc[Av_Fl[0][f-1], 'EC_Oversold']} 

# Initialize count for each class
class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
filtered_pass_data_c1_f1['Assigned Class'] = filtered_pass_data_c1_f1.apply(assign_class, axis=1)

In [382]:
#Alternate flight 2
#f = 2
#filt = (aff_pax_c1['Alt_Fli'] == Av_Fl[0][f-1])
#filtered_c1_f2 = aff_pax_c1.loc[filt]

#filt = (aff_pax_c1['Alt_Fli'] == Av_Fl[0][f-1])
#filtered_pass_data_c1_f2 = aff_pax_c1.loc[filt]

In [383]:
#Assigning class based on PNR scores
# Sort the DataFrame based on 'Score' in descending order
#filtered_pass_data_c1_f2 = filtered_pass_data_c1_f2.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
#class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
#class_limits = {
#    'FirstClass': df2.loc[Av_Fl[0][f-1], 'FC_Oversold'], 
#    'BusinessClass': df2.loc[Av_Fl[0][f-1], 'BC_Oversold'], 
#    'PremiumEconomyClass': df2.loc[Av_Fl[0][f-1], 'PC_Oversold'], 
#    'EconomyClass': df2.loc[Av_Fl[0][f-1], 'EC_Oversold']} 

# Initialize count for each class
#class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
#filtered_pass_data_c1_f2['Assigned Class'] = filtered_pass_data_c1_f2.apply(assign_class, axis=1)

In [384]:
#Passengers who were alloted the same class in flight 1
filt = (filtered_pass_data_c1_f1['Class'] == filtered_pass_data_c1_f1['Assigned Class'])
same_allotment_c1_f1 = filtered_pass_data_c1_f1.loc[filt]  

#Passengers who were alloted the same class in flight 2
#filt = (filtered_pass_data_c1_f2['Class'] == filtered_pass_data_c1_f2['Assigned Class'])
#same_allotment_2 = filtered_pass_data_c1_f1.loc[filt]

In [385]:
#Passengers who were downgraded 

In [386]:
#Flight 1

In [387]:
filt1 = (filtered_pass_data_c1_f1['Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c1_f1['Assigned Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c1_f1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c1_f1['Assigned Class'] == 'EconomyClass')
FC_dn_c1_f1 = filtered_pass_data_c1_f1.loc[filt1 & (filt2 | filt3| filt4)]

filt2 = (filtered_pass_data_c1_f1['Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c1_f1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c1_f1['Assigned Class'] == 'EconomyClass')
BC_dn_c1_f1 = filtered_pass_data_c1_f1.loc[filt2 & (filt3 | filt4)]

filt3 = (filtered_pass_data_c1_f1['Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c1_f1['Assigned Class'] == 'EconomyClass')
PC_dn_c1_f1 = filtered_pass_data_c1_f1.loc[filt3 & (filt4)]

In [388]:
#Flight 2

In [389]:
#filt1 = (filtered_pass_data_c1_f2['Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c1_f2['Assigned Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c1_f2['Assigned Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c1_f2['Assigned Class'] == 'EconomyClass')
#FC_dn_c1_f2 = filtered_pass_data_c1_f2.loc[filt1 & (filt2 | filt3| filt4)]

#filt2 = (filtered_pass_data_c1_f2['Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c1_f2['Assigned Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c1_f2['Assigned Class'] == 'EconomyClass')
#BC_dn_c1_f2 = filtered_pass_data_c1_f2.loc[filt2 & (filt3 | filt4)]

#filt3 = (filtered_pass_data_c1_f2['Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c1_f2['Assigned Class'] == 'EconomyClass')
#PC_dn_c1_f2 = filtered_pass_data_c1_f2.loc[filt3 & (filt4)]

In [390]:
#Passengers who were upgraded 

In [391]:
#Flight 1

In [392]:
filt1 = (filtered_pass_data_c1_f1['Assigned Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c1_f1['Assigned Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c1_f1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c1_f1['Class'] == 'EconomyClass')
EC_up_c1_f1 = filtered_pass_data_c1_f1.loc[filt4 & (filt1 | filt2| filt3)]

filt1 = (filtered_pass_data_c1_f1['Assigned Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c1_f1['Assigned Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c1_f1['Class'] == 'PremiumEconomyClass')
PC_up_c1_f1 = filtered_pass_data_c1_f1.loc[filt3 & (filt1 | filt2)]

filt1 = (filtered_pass_data_c1_f1['Assigned Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c1_f1['Class'] == 'BusinessClass')
BC_up_c1_f1 = filtered_pass_data_c1_f1.loc[filt2 & filt1]

In [393]:
#Flight 2

In [394]:
#filt1 = (filtered_pass_data_c1_f2['Assigned Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c1_f2['Assigned Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c1_f2['Assigned Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c1_f2['Class'] == 'EconomyClass')
#EC_up_c1_f2 = filtered_pass_data_c1_f2.loc[filt4 & (filt1 | filt2| filt3)]

#filt1 = (filtered_pass_data_c1_f2['Assigned Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c1_f2['Assigned Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c1_f2['Class'] == 'PremiumEconomyClass')
#PC_up_c1_f2 = filtered_pass_data_c1_f2.loc[filt3 & (filt1 | filt2)]

#filt1 = (filtered_pass_data_c1_f2['Assigned Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c1_f2['Class'] == 'BusinessClass')
#BC_up_c1_f2 = filtered_pass_data_c1_f2.loc[filt2 & filt1]

In [395]:
print(f"{len(same_allotment_c1_f1)} passengers were allotted the same class they applied for")
print(f"{len(FC_dn_c1_f1) + len(BC_dn_c1_f1) + len(PC_dn_c1_f1)} passengers were downgraded")
print(f"{len(EC_up_c1_f1) + len(PC_up_c1_f1) + len(BC_up_c1_f1)} passengers were upgraded")

1 passengers were allotted the same class they applied for
10 passengers were downgraded
8 passengers were upgraded


In [396]:
#C2

In [397]:
#Alternate flight 1
f = 1
filt = (aff_pax_c2['Alt_Fli'] == Av_Fl[1][f-1])
filtered_c2_f1 = aff_pax_c2.loc[filt]

filt = (aff_pax_c2['Alt_Fli'] == Av_Fl[1][f-1])
filtered_pass_data_c2_f1 = aff_pax_c2.loc[filt]

In [398]:
#Assigning class based on PNR scores
# Sort the DataFrame based on 'Score' in descending order
filtered_pass_data_c2_f1 = filtered_pass_data_c2_f1.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
class_limits = {
    'FirstClass': df2.loc[Av_Fl[1][f-1], 'FC_Oversold'], 
    'BusinessClass': df2.loc[Av_Fl[1][f-1], 'BC_Oversold'], 
    'PremiumEconomyClass': df2.loc[Av_Fl[1][f-1], 'PC_Oversold'], 
    'EconomyClass': df2.loc[Av_Fl[1][f-1], 'EC_Oversold']} 

# Initialize count for each class
class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
filtered_pass_data_c2_f1['Assigned Class'] = filtered_pass_data_c2_f1.apply(assign_class, axis=1)

In [399]:
#Alternate flight 2
#f = 2
#filt = (aff_pax_c2['Alt_Fli'] == Av_Fl[1][f-1])
#filtered_c2_f2 = aff_pax_c2.loc[filt]

#filt = (aff_pax_c2['Alt_Fli'] == Av_Fl[1][f-1])
#filtered_pass_data_c2_f2 = aff_pax_c2.loc[filt]

In [400]:
#Assigning class based on PNR scores
# Sort the DataFrame based on 'Score' in descending order
#filtered_pass_data_c2_f2 = filtered_pass_data_c2_f2.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
#class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
#class_limits = {
#    'FirstClass': df2.loc[Av_Fl[1][f-1], 'FC_Oversold'], 
#    'BusinessClass': df2.loc[Av_Fl[1][f-1], 'BC_Oversold'], 
#    'PremiumEconomyClass': df2.loc[Av_Fl[1][f-1], 'PC_Oversold'], 
#    'EconomyClass': df2.loc[Av_Fl[1][f-1], 'EC_Oversold']} 

# Initialize count for each class
#class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
#filtered_pass_data_c2_f2['Assigned Class'] = filtered_pass_data_c2_f2.apply(assign_class, axis=1)

In [401]:
#Passengers who were alloted the same class in flight 1
filt = (filtered_pass_data_c2_f1['Class'] == filtered_pass_data_c2_f1['Assigned Class'])
same_allotment_c2_f1 = filtered_pass_data_c2_f1.loc[filt]  

#Passengers who were alloted the same class in flight 2
#filt = (filtered_pass_data_c1_f2['Class'] == filtered_pass_data_c1_f2['Assigned Class'])
#same_allotment_2 = filtered_pass_data_c1_f1.loc[filt]

Passengers who were downgraded 

In [403]:
#Flight 1

In [404]:
filt1 = (filtered_pass_data_c2_f1['Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c2_f1['Assigned Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c2_f1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c2_f1['Assigned Class'] == 'EconomyClass')
FC_dn_c2_f1 = filtered_pass_data_c2_f1.loc[filt1 & (filt2 | filt3| filt4)]

filt2 = (filtered_pass_data_c2_f1['Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c2_f1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c2_f1['Assigned Class'] == 'EconomyClass')
BC_dn_c2_f1 = filtered_pass_data_c2_f1.loc[filt2 & (filt3 | filt4)]

filt3 = (filtered_pass_data_c2_f1['Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c2_f1['Assigned Class'] == 'EconomyClass')
PC_dn_c2_f1 = filtered_pass_data_c2_f1.loc[filt3 & (filt4)]

In [405]:
#Flight 2

In [406]:
#filt1 = (filtered_pass_data_c2_f2['Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c2_f2['Assigned Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c2_f2['Assigned Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c2_f2['Assigned Class'] == 'EconomyClass')
#FC_dn_c2_f2 = filtered_pass_data_c1_f2.loc[filt1 & (filt2 | filt3| filt4)]

#filt2 = (filtered_pass_data_c2_f2['Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c2_f2['Assigned Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c2_f2['Assigned Class'] == 'EconomyClass')
#BC_dn_c2_f2 = filtered_pass_data_c2_f2.loc[filt2 & (filt3 | filt4)]

#filt3 = (filtered_pass_data_c2_f2['Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c2_f2['Assigned Class'] == 'EconomyClass')
#PC_dn_c2_f2 = filtered_pass_data_c2_f2.loc[filt3 & (filt4)]

Passengers who were upgraded 

In [408]:
#Flight 1

In [409]:
filt1 = (filtered_pass_data_c2_f1['Assigned Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c2_f1['Assigned Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c2_f1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filtered_pass_data_c2_f1['Class'] == 'EconomyClass')
EC_up_c2_f1 = filtered_pass_data_c2_f1.loc[filt4 & (filt1 | filt2| filt3)]

filt1 = (filtered_pass_data_c2_f1['Assigned Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c2_f1['Assigned Class'] == 'BusinessClass')
filt3 = (filtered_pass_data_c2_f1['Class'] == 'PremiumEconomyClass')
PC_up_c2_f1 = filtered_pass_data_c2_f1.loc[filt3 & (filt1 | filt2)]

filt1 = (filtered_pass_data_c2_f1['Assigned Class'] == 'FirstClass')
filt2 = (filtered_pass_data_c2_f1['Class'] == 'BusinessClass')
BC_up_c2_f1 = filtered_pass_data_c2_f1.loc[filt2 & filt1]

In [410]:
#Flight 2

In [411]:
#filt1 = (filtered_pass_data_c2_f2['Assigned Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c2_f2['Assigned Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c2_f2['Assigned Class'] == 'PremiumEconomyClass')
#filt4 = (filtered_pass_data_c2_f2['Class'] == 'EconomyClass')
#EC_up_c2_f2 = filtered_pass_data_c2_f2.loc[filt4 & (filt1 | filt2| filt3)]

#filt1 = (filtered_pass_data_c2_f2['Assigned Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c2_f2['Assigned Class'] == 'BusinessClass')
#filt3 = (filtered_pass_data_c2_f2['Class'] == 'PremiumEconomyClass')
#PC_up_c2_f2 = filtered_pass_data_c2_f2.loc[filt3 & (filt1 | filt2)]

#filt1 = (filtered_pass_data_c2_f2['Assigned Class'] == 'FirstClass')
#filt2 = (filtered_pass_data_c2_f2['Class'] == 'BusinessClass')
#BC_up_c2_f2 = filtered_pass_data_c2_f2.loc[filt2 & filt1]

In [412]:
print(f"{len(same_allotment_c2_f1)} passengers were allotted the same class they applied for")
print(f"{len(FC_dn_c2_f1) + len(BC_dn_c2_f1) + len(PC_dn_c2_f1)} passengers were downgraded")
print(f"{len(EC_up_c2_f1) + len(PC_up_c2_f1) + len(BC_up_c2_f1)} passengers were upgraded")

13 passengers were allotted the same class they applied for
19 passengers were downgraded
27 passengers were upgraded


In [413]:
print(f"{len(same_allotment_c1_f1) + len(same_allotment_c2_f1)} passengers were allotted the same class they applied for")
print(f"{len(FC_dn_c1_f1) + len(BC_dn_c1_f1) + len(PC_dn_c1_f1) + len(FC_dn_c2_f1) + len(BC_dn_c2_f1) + len(PC_dn_c2_f1)} passengers were downgraded")
print(f"{len(EC_up_c1_f1) + len(PC_up_c1_f1) + len(BC_up_c1_f1) + len(EC_up_c2_f1) + len(PC_up_c2_f1) + len(BC_up_c2_f1)} passengers were upgraded")

14 passengers were allotted the same class they applied for
29 passengers were downgraded
35 passengers were upgraded
